![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, I will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. I will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

I've been provided with five datasets to investigate:
* `info.csv`
* `finance.csv`
* `reviews.csv`
* `traffic.csv`
* `brands.csv`

The following are the questions we are going to answer:

## What is the volume of products and average revenue for Adidas and Nike products based on price quartiles?

* Label products priced up to quartile one as `"Budget"`, quartile 2 as `"Average"`, quartile 3 as `"Expensive"`, and quartile 4 as `"Elite"`.
* Store as a `pandas` DataFrame called `adidas_vs_nike` containing the following columns: `"brand"`, `"price_label"`, `"num_products"`, and `"mean_revenue"`.

## Do any differences exist between the word count of a product's description and its mean rating?

* Store the results as a `pandas` DataFrame called `description_lengths` containing the following columns: `"description_length"`, `"mean_rating"`, `"num_reviews"`.

## How does the volume of products and median revenue vary between clothing and footwear?

* **Create** a `pandas` DataFrame called `product_types` containing the following columns: `"num_clothing_products"`, `"median_clothing_revenue"`, `"num_footwear_products"`, `"median_footwear_revenue"`.

In [47]:
# Start coding here... 
import pandas as pd


In [48]:
# Read in the data
info = pd.read_csv("info.csv")
finance = pd.read_csv("finance.csv")
reviews = pd.read_csv("reviews.csv")
traffic = pd.read_csv("traffic.csv")
brands = pd.read_csv("brands.csv")

In [49]:



# Merge the data
merged_df = info.merge(finance, on="product_id", how="outer")
merged_df = merged_df.merge(reviews, on="product_id", how="outer")
merged_df = merged_df.merge(traffic, on="product_id", how="outer")
merged_df = merged_df.merge(brands, on="product_id", how="outer")



In [50]:
# Remove null values
merged_df.dropna(inplace=True)

In [51]:
# Add price labels based on listing_price quartiles
merged_df["price_label"] = pd.qcut(merged_df["listing_price"], q=4, labels=["Budget", "Average", "Expensive", "Elite"])



In [52]:
# Group by brand and price_label to get volume and mean revenue
adidas_vs_nike = merged_df.groupby(["brand", "price_label"], as_index=False).agg(
    num_products=("price_label", "count"), 
    mean_revenue=("revenue", "mean")
).round(2).reset_index(drop=True)


In [53]:
# Find the largest description_length
max(merged_df["description"].str.len())

# Store the length of each description
merged_df["description_length"] = merged_df["description"].str.len()

In [54]:
# Upper description length limits
lengthes = [0, 100, 200, 300, 400, 500, 600, 700]

# Description length labels
labels = ["100", "200", "300", "400", "500", "600", "700"]

In [55]:
# Cut into bins
merged_df["description_length"] = pd.cut(merged_df["description_length"], bins=lengthes, labels=labels)

# Group by the bins
description_lengths = merged_df.groupby("description_length", as_index=False).agg(
    mean_rating=("rating", "mean"), 
    num_reviews=("reviews", "count")
).round(2)




In [56]:
# List of footwear keywords
mylist = "shoe*|trainer*|foot*"

# Filter for footwear products
shoes = merged_df[merged_df["description"].str.contains(mylist)]

# Filter for clothing products
clothing = merged_df[~merged_df.isin(shoes["product_id"])]

# Remove null product_id values from clothing DataFrame
clothing.dropna(inplace=True)

In [57]:
# Create product_types DataFrame
product_types = pd.DataFrame({"num_clothing_products": len(clothing), 
                              "median_clothing_revenue": clothing["revenue"].median(), 
                              "num_footwear_products": len(shoes), 
                              "median_footwear_revenue": shoes["revenue"].median()}, 
                              index=[0])

In [58]:
product_types

,num_clothing_products,median_clothing_revenue,num_footwear_products,median_footwear_revenue
0,439,683.73,2434,3073.3
